In [1]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor

from datetime import timedelta


In [4]:
df = pd.read_csv("dataset.csv")

In [5]:
df['order_date'] = pd.to_datetime(df['order_date'])

daily_demand = df.groupby(['warehouse_id', 'item_name', 'order_date'])['quantity'].sum().reset_index()

full_data = []
for (w_id, item), group in daily_demand.groupby(['warehouse_id', 'item_name']):
    date_range = pd.date_range(start=group['order_date'].min(), end=group['order_date'].max())
    group = group.set_index('order_date').reindex(date_range).fillna(0).rename_axis('order_date').reset_index()
    group['warehouse_id'] = w_id
    group['item_name'] = item
    full_data.append(group)

full_df = pd.concat(full_data, ignore_index=True)

full_df['dayofweek'] = full_df['order_date'].dt.dayofweek
full_df['month'] = full_df['order_date'].dt.month
full_df['year'] = full_df['order_date'].dt.year
full_df['quantity'] = full_df['quantity'].astype(float)


<ipython-input-5-fd08ff0bc6e3>:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['order_date'] = pd.to_datetime(df['order_date'])


In [13]:

cutoff_date = full_df['order_date'].max() - timedelta(days=14)
train_df = full_df[full_df['order_date'] <= cutoff_date]
test_df = full_df[full_df['order_date'] > cutoff_date]

predictions = []
features = ['dayofweek', 'month', 'year' ]

for (w_id, item), group in train_df.groupby(['warehouse_id', 'item_name']):
    test_group = test_df[(test_df['warehouse_id'] == w_id) & (test_df['item_name'] == item)]
    if test_group.empty:
        continue

    X_train = group[features]
    y_train = group['quantity']
    X_test = test_group[features]
    y_test = test_group['quantity']

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    test_group = test_group.copy()
    test_group['predicted_quantity'] = y_pred
    predictions.append(test_group)


In [9]:

forecast_df = pd.concat(predictions, ignore_index=True)
inventory_forecast = forecast_df.groupby(['warehouse_id', 'item_name'])['predicted_quantity'].sum().reset_index()
inventory_forecast.rename(columns={'predicted_quantity': 'total_quantity_next_14_days'}, inplace=True)


In [10]:

inventory_forecast.to_csv('14_day_inventory_forecast.csv', index=False)
print("Inventory forecast saved as '14_day_inventory_forecast.csv'")

✅ Inventory forecast saved as '14_day_inventory_forecast.csv'
